In [6]:
import requests
import configparser

In [7]:
class SessionEarthData(requests.Session):
    AUTH_HOST = 'urs.earthdata.nasa.gov'

    def __init__(self, username, password):
        super().__init__()
        self.auth = (username, password)

    def rebuild_auth(self, prepared_request, response):
        headers = prepared_request.headers
        url = prepared_request.url
        if 'Authorization' in headers:
            original_parsed = requests.utils.urlparse(response.request.url)
            redirect_parsed = requests.utils.urlparse(url)
            if (original_parsed.hostname != redirect_parsed.hostname) and \
                    redirect_parsed.hostname != self.AUTH_HOST and \
                    original_parsed.hostname != self.AUTH_HOST:
                del headers['Authorization']
        return

In [8]:
config = configparser.ConfigParser()
config.read('user.config')
username= config['user']['user']
password = config['user']['pwd']
session = SessionEarthData(username=username, password=password)

In [9]:
session

In [11]:
url = 'https://opendap.earthdata.nasa.gov/collections/C1996881146-POCLOUD/granules/20220102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.dmr'
ret = session.get(url)

In [14]:
with open('data.dmr', 'wt') as f:
    f.write(ret.text)